### **피보나치 수열(Fibonacci Sequence)의 효율적 계산에 대한 분석 보고서**

### **목차**

- **1. 서론**
    - 1.1. 보고서의 목표 및 구성

- **2. 기본적인 접근법과 한계**
    - 2.1. 단순 재귀 호출: $O(2^n)$
    - 2.2. 동적 계획법 (Dynamic Programming): $O(n)$
        - A. 하향식 접근 (Top-Down with Memoization)
        - B. 상향식 접근 (Bottom-Up with Tabulation)

- **3. 수학적 접근을 통한 성능 개선: $O(\log n)$**
    - 3.1. 상태 전이 행렬의 도입
    - 3.2. 행렬 거듭제곱과 피보나치 수열의 관계 증명
    - 3.3. 분할 정복을 이용한 행렬의 빠른 거듭제곱

- **4. $O(\log n)$ 알고리즘 구현 (Python)**
    - 4.1. 행렬 거듭제곱 함수 구현
    - 4.2. 최종 피보나치 함수

- **5. 성능 비교 및 결론**
    - 5.1. 시간 복잡도 비교 요약
    - 5.2. 최종 결론

---

### **1. 서론**


#### **1.1. 보고서의 목표 및 구성**

본 보고서의 목표는 피보나치 수열을 계산하는 다양한 알고리즘을 분석하고, 그 시간 복잡도의 한계를 점진적으로 개선해 나가는 과정을 보이는 것이다. 단순한 재귀 구현($O(2^n)$)에서 시작하여 동적 계획법을 통한 선형 시간($O(n)$) 최적화를 거쳐, 최종적으로 행렬의 거듭제곱과 분할 정복을 활용한 로그 시간($O(\log n)$) 알고리즘까지 탐구한다.

### **2. 기본적인 접근법과 한계**

#### **2.1. 단순 재귀 호출: $O(2^n)$**
**[그림 1] 단순 재귀**
![d1](../image/O(n^2).png) 
피보나치 수열의 정의를 그대로 코드로 옮긴 가장 직관적인 방법이다.

```python
def fib_recursive(n):
    if n <= 1:
        return n
    return fib_recursive(n-1) + fib_recursive(n-2)
```

-   **문제점**: `fib(n-1)`과 `fib(n-2)`를 계산하는 과정에서 동일한 부분 문제(예: `fib(n-3)`)가 수없이 중복 호출된다. 이로 인해 호출 스택이 기하급수적으로 증가하며 시간 복잡도는 $O(2^n)$에 달한다. `n`이 40-50만 되어도 계산이 거의 불가능해진다.

#### **2.2. 동적 계획법 (Dynamic Programming): $O(n)$**
**[그림 2] 동적계획법**
![d2](../image/O(n).png) 
단순 재귀의 '중복 호출' 문제를 해결하기 위해 동적 계획법을 사용한다. 한 번 계산한 값은 저장해두고 재사용하여 계산 횟수를 획기적으로 줄인다.

**A. 하향식 접근 (Top-Down with Memoization)**

재귀 구조를 유지하되, 계산 결과를 배열(memo)에 저장하여 중복 계산을 방지한다.

```python
# 메모이제이션을 위한 배열
memo = {0: 0, 1: 1}

def fib_top_down(n):
    if n in memo:
        return memo[n]
    
    memo[n] = fib_top_down(n-1) + fib_top_down(n-2)
    return memo[n]

print(fib_top_down(99))
```

**B. 상향식 접근 (Bottom-Up with Tabulation)**

반복문을 사용하여 가장 작은 문제부터 차례로 계산하여 테이블(dp_table)을 채워나간다.

```python
def fib_bottom_up(n):
    if n <= 1:
        return n
        
    dp_table = [0] * (n + 1)
    dp_table[1] = 1
    
    for i in range(2, n + 1):
        dp_table[i] = dp_table[i-1] + dp_table[i-2]
        
    return dp_table[n]

print(fib_bottom_up(99))
```

-   **결론**: 동적 계획법을 통해 시간 복잡도를 $O(n)$으로 크게 개선했다. 이는 대부분의 경우 충분히 효율적이지만, `n`이 수십억 단위로 커질 경우 여전히 한계가 있다.

### **3. 수학적 접근을 통한 성능 개선: $O(\log n)$**
**[그림 3] 단순 재귀**
![d3](../image/O(logn).png) 
시간 복잡도를 $O(n)$에서 더 단축하기 위해 피보나치 수열의 점화식을 행렬 형태로 변환한다.

#### **3.1. 상태 전이 행렬의 도입**

피보나치 수열의 한 상태에서 다음 상태로 넘어가는 관계는 다음과 같은 행렬 곱으로 표현할 수 있다.
$$
\begin{bmatrix} F_{n+1} \\ F_n \end{bmatrix} = \begin{bmatrix} 1 & 1 \\ 1 & 0 \end{bmatrix} \begin{bmatrix} F_n \\ F_{n-1} \end{bmatrix}
$$
여기서 행렬 $A = \begin{bmatrix} 1 & 1 \\ 1 & 0 \end{bmatrix}$를 **상태 전이 행렬**이라 부른다. 이 관계를 반복 적용하면 다음 식을 유도할 수 있다.
$$
\begin{bmatrix} F_{n+1} \\ F_n \end{bmatrix} = \begin{bmatrix} 1 & 1 \\ 1 & 0 \end{bmatrix}^n \begin{bmatrix} F_1 \\ F_0 \end{bmatrix}
$$
여기서 $F_0=0, F_1=1$을 사용하면, $F_n$을 구하는 문제는 결국 행렬 $A^n$을 효율적으로 계산하는 문제로 귀결된다.

#### **3.2. 행렬 거듭제곱과 피보나치 수열의 관계 증명**

다음 식이 성립한다.: $A^n = \begin{bmatrix} 1 & 1 \\ 1 & 0 \end{bmatrix}^n = \begin{bmatrix} F_{n+1} & F_n \\ F_n & F_{n-1} \end{bmatrix}$ 

**증명 (수학적 귀납법):**

1. **기저 사례 ($n=1$)**  
   $$
   A^1 = \begin{bmatrix} 1 & 1 \\ 1 & 0 \end{bmatrix}
   $$
   피보나치 항으로 표현하면 $F_2=1$, $F_1=1$, $F_0=0$ 이므로,  
   $$
   \begin{bmatrix} F_2 & F_1 \\ F_1 & F_0 \end{bmatrix} = \begin{bmatrix} 1 & 1 \\ 1 & 0 \end{bmatrix}
   $$
   따라서 $n=1$일 때 성립합니다.

2. **귀납적 가정**  
   $n=k$일 때  
   $$
   A^k = \begin{bmatrix} F_{k+1} & F_k \\ F_k & F_{k-1} \end{bmatrix}
   $$  
   가 성립한다고 가정합니다.

3. **귀납 단계**  
   $n=k+1$일 때 성립함을 보이면 됩니다.  
   $$
   A^{k+1} = A^k \cdot A = \begin{bmatrix} F_{k+1} & F_k \\ F_k & F_{k-1} \end{bmatrix} \begin{bmatrix} 1 & 1 \\ 1 & 0 \end{bmatrix}
   $$
   행렬 곱셈을 수행하면,  
   $$
   = \begin{bmatrix} F_{k+1} + F_k & F_{k+1} \\ F_k + F_{k-1} & F_k \end{bmatrix}
   $$
   피보나치 점화식 ($F_{n} = F_{n-1} + F_{n-2}$)을 적용하여 정리하면,  
   $$
   = \begin{bmatrix} F_{k+2} & F_{k+1} \\ F_{k+1} & F_k \end{bmatrix}
   $$
   이는 $n=k+1$일 때의 목표 형태와 일치합니다.

따라서 모든 자연수 $n$에 대해 다음 식이 성립하니다. 
$$
A^n = \begin{bmatrix} F_{n+1} & F_n \\ F_n & F_{n-1} \end{bmatrix}
$$  


#### **3.3. 분할 정복을 이용한 행렬의 빠른 거듭제곱**

$A^n$을 계산할 때 $A$를 $n-1$번 곱하면 여전히 $O(n)$이다. 하지만 **분할 정복(Divide and Conquer)**을 이용하면 $O(\log n)$만에 계산할 수 있다.
-   **n이 짝수일 때**: $A^n = A^{n/2} \cdot A^{n/2}$
-   **n이 홀수일 때**: $A^n = A \cdot A^{n-1} = A \cdot A^{(n-1)/2} \cdot A^{(n-1)/2}$

이 재귀적인 관계를 통해 행렬 곱셈 횟수를 대폭 줄일 수 있다.

### **4. $O(\log n)$ 알고리즘 구현 (Python)**

`numpy` 라이브러리를 사용하면 행렬 연산을 간결하게 구현할 수 있다.

#### **4.1. 행렬 거듭제곱 함수 구현**
```python
import numpy as np

# 분할 정복을 이용한 행렬의 빠른 거듭제곱 함수
def matrix_power(matrix, n):
    # 기저 사례: n=1일 때 행렬 자신을 반환
    if n == 1:
        return matrix
    
    # n이 짝수인 경우
    if n % 2 == 0:
        half = matrix_power(matrix, n // 2)
        # numpy의 @ 연산자 또는 np.matmul()을 사용한 행렬 곱셈
        return half @ half
    # n이 홀수인 경우
    else:
        half = matrix_power(matrix, (n - 1) // 2)
        return matrix @ half @ half
```

#### **4.2. 최종 피보나치 함수**

```python
def fib_matrix(n):
    if n <= 1:
        return n
    
    # 상태 전이 행렬 정의
    base_matrix = np.array([[1, 1], [1, 0]], dtype=object) # 큰 수를 다루기 위해 dtype=object
    
    # F_n은 A^(n-1)의 [0,0] 원소
    result_matrix = matrix_power(base_matrix, n - 1)
    
    return result_matrix[0][0]

# 100번째 피보나치 수 계산
num = 100
print(f"{num}번째 피보나치 수: {fib_matrix(num)}")
```


### **5. 성능 비교 및 결론**

#### **5.1. 실험 설계**

이론적 시간 복잡도의 차이가 실제 실행 시간에 어떤 영향을 미치는지 측정하기 위해 다음과 같이 실험을 설계한다.

-   **테스트 환경**: Google Colab (Python, `numpy`, `matplotlib`)
-   **측정 대상 알고리즘**:
    1.  단순 재귀 (`fib_recursive`)
    2.  동적 계획법 (`fib_dp_bottom_up`)
    3.  행렬 거듭제곱 (`fib_matrix`)
-   **입력 값(n)**:
    -   **소규모 그룹 (n = 10, 20, 30, 35)**: 모든 알고리즘의 성능을 비교한다.
    -   **대규모 그룹 (n = 1000, 10000, 100000, 500000)**: 실행 가능한 DP와 행렬 알고리즘의 확장성을 비교한다.
-   **측정 지표**: 각 알고리즘의 `n`에 대한 실행 시간을 `time.perf_counter`를 이용해 측정한다.

#### **5.2. 실험 결과 및 시각화**

실험 결과는 아래의 그래프와 표와 같다. 성능 차이가 매우 크므로 그래프의 x축과 y축은 모두 로그 스케일(log scale)을 적용하여 시각화했다.

**[그림 4] 피보나치 알고리즘 성능 비교 그래프**
![divide](../image/fibP.png) 


위 그래프는 각 알고리즘의 시간 복잡도 특성을 명확히 보여준다.
-   **단순 재귀 (빨간색)**: `n`이 조금만 증가해도 실행 시간이 급격히 치솟으며, $O(2^n)$의 폭발적인 증가율을 보인다.
-   **동적 계획법 (파란색)**: 로그 스케일 그래프에서 거의 직선 형태로 나타나며, $O(n)$의 안정적인 선형 증가율을 보인다.
-   **행렬 거듭제곱 (초록색)**: `n`이 크게 증가함에도 실행 시간이 거의 변하지 않는, 매우 완만한 $O(\log n)$의 증가율을 보인다.

**[표 1] 알고리즘별 실행 시간 측정 결과**
![divide](../image/fibP2.png) 

위 표는 실제 측정된 실행 시간을 보여준다. `n=35`에서 이미 단순 재귀는 DP 방식보다 수십만 배 이상 느리다. 대규모 `n`에서는 DP 방식도 수 초가 걸리는 반면, 행렬 거듭제곱 방식은 거의 즉시 결과를 반환하여 압도적인 성능 차이를 입증한다.

#### **5.3. 결과 분석**

-   **단순 재귀의 한계**: `[그림 1]`에서 시각화된 것처럼, 중복된 함수 호출이 성능 저하의 주된 원인이다. 실험 결과는 이론적 복잡도인 $O(2^n)$가 실제 환경에서 얼마나 비효율적인지를 명확히 보여준다.

-   **동적 계획법의 효율성**: `[그림 2]`의 선형적 계산 구조 덕분에, DP는 중복 계산 문제를 완벽히 해결한다. `n`이 수십만 단위까지는 실용적인 성능을 제공하며, 대부분의 문제 상황에서 효과적인 해결책이다.

-   **행렬 거듭제곱의 압도적 성능**: `[그림 3]`의 분할 정복 원리를 통해, `n`의 크기에 거의 영향을 받지 않는 $O(\log n)$의 성능을 달성했다. 이는 문제를 다른 관점(상태 전이 행렬)으로 재정의하고, 수학적 도구(분할 정복)를 적용한 결과이다. `n`이 매우 큰 범위(수십억 이상)를 다뤄야 하는 경쟁 프로그래밍이나 암호학 등의 분야에서 필수적인 접근법이다.

#### **5.4. 최종 결론**

본 보고서의 실험을 통해 피보나치 수열을 계산하는 각 알고리즘의 이론적 시간 복잡도가 실제 성능에 어떻게 반영되는지를 확인했다. 단순 재귀의 $O(2^n)$ 비효율성은 동적 계획법을 통해 $O(n)$으로 개선될 수 있으며, 여기서 더 나아가 문제의 수학적 구조를 활용한 행렬 거듭제곱 방식은 $O(\log n)$이라는 최적의 성능을 달성할 수 있다. 결론적으로, 동일한 문제라도 어떤 알고리즘을 설계하고 적용하는지에 따라 성능이 극적으로 달라질 수 있음을 알 수 있다.